### 完全情報ゲームの例: $\circ \times$ ゲーム (Tic-tac-toe)
#### 前提
- $\times$ のプレイヤーが先手，$\circ$ のプレイヤーが後手とする
- 各マスを次の番号で区別する  


  |<!-- -->  |<!-- -->  |<!-- -->  |
  |---|---|---|
  |0  |1  |2  |
  |3  |4  |5  |
  |6  |7  |8  |  
   

- 各プレイヤーの手番をマスの番号の list/set で表現する。  
  例えば， X のプレイヤーの手番が [0, 1, 2] であれば次の状態に対応する。  
  (ただし，"*" は X，O，空白のいずれかを表すものとする)


  
  |<!-- -->  |<!-- -->  |<!-- -->  |
  |---|---|---|
  |X  |X  |X  |
  |*  |*  |*  |
  |*  |*  |*  |

#### 引き分け状態の個数を数える
- 引き分け(draw)とは，9マスすべてが埋まった状態にも関わらず，X も O も勝利していない状態である
- 最終的なボードの状態が同じでも、途中の手番が異なれば別の結果として数える

In [1]:
import itertools
import numpy as np

NUM_PLAYERS = 2
win_seq = [{0, 1, 2}, {3, 4, 5}, {6, 7, 8}, # horizontal
           {0, 3, 6}, {1, 4, 7}, {2, 5, 8}, # vertical
           {0, 4, 8}, {2, 4, 6},] # diagonal 
histories = [p for p in itertools.permutations(range(9))]
moves_by_x = (set(h[0::NUM_PLAYERS]) for h in histories)
moves_by_o = (set(h[1::NUM_PLAYERS]) for h in histories)
idx_won_by_x = [any(s.issubset(m) for s in win_seq) for m in moves_by_x]
idx_won_by_o = [any(s.issubset(m) for s in win_seq) for m in moves_by_o]
idx_won_by_x_or_o = np.logical_or(idx_won_by_x, idx_won_by_o)

print('Draws: ', len(histories) - np.count_nonzero(idx_won_by_x_or_o))
# print('Draws: ', len(histories) - len(np.array(histories)[idx_won_by_x_or_o]))

Draws:  46080


##### 解説:
- 以下の解説のセルを正常に実行するには Notebook の最初から順にセルを実行しておく必要がある。

[`itertools.permutations(t)`](https://docs.python.org/ja/3/library/itertools.html#itertools.permutations) で `t` の要素を全て並べるときにありうる全ての順列を生成できる

In [2]:
import itertools as it

tuple(it.permutations(('A', 'B', 'C')))

(('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A'))

`itertools.permutations(range(9))` で，9マスすべてが埋まるまでにありうる全ての手番を生成できる。  

（ただし，その中には，途中で決着がついているものも含まれる）

In [3]:
NUM_CELLS = 9
NUM_COLS = 3

def print_board(histories):
    txt = ''
    for i in range(NUM_CELLS):
        symbol = ' '
        if i in histories[0]:
            symbol = 'X'
        elif i in histories[1]:
            symbol = 'O'
        txt += symbol + ' '
        if i % NUM_COLS == (NUM_COLS - 1):
            txt += '\n'
    print(txt)

def separate_and_print_board(h):
    print('h = ', h)
    m_by_x = h[0::NUM_PLAYERS]
    m_by_o = h[1::NUM_PLAYERS]
    print('m_by_x = ', m_by_x)
    print('m_by_o = ', m_by_o)
    print_board([m_by_x, m_by_o])


histories = [p for p in itertools.permutations(range(9))]
separate_and_print_board(histories[0])
separate_and_print_board(histories[1234])

h =  (0, 1, 2, 3, 4, 5, 6, 7, 8)
m_by_x =  (0, 2, 4, 6, 8)
m_by_o =  (1, 3, 5, 7)
X O X 
O X O 
X O X 

h =  (0, 1, 3, 7, 4, 5, 8, 2, 6)
m_by_x =  (0, 3, 4, 8, 6)
m_by_o =  (1, 7, 5, 2)
X O O 
X X O 
X O X 



シーケンス `a` に対して `a[start:end:step]` は スライス表記([slicing](https://docs.python.org/ja/3/reference/expressions.html#slicings))であり，インデックスが start, start + step, start + 2*step, $\ldots$
である要素を取り出して，新たなシーケンスを作る。ここで end は省略可能。

In [4]:
a = tuple(range(9))
print('a ==', repr(a))
print('a[0::2] ==', repr(a[0::2]))
print('a[1::2] ==', repr(a[1::2]))

a == (0, 1, 2, 3, 4, 5, 6, 7, 8)
a[0::2] == (0, 2, 4, 6, 8)
a[1::2] == (1, 3, 5, 7)


- 最終状態までの X  の一連の手番に，勝利条件を満たす一連の手番(win_seq) のどれか一つが含まれていれば X が勝利している。
  - O についても同様のことがいえる。
- これを確認するには [set 型](https://docs.python.org/ja/3/library/stdtypes.html#set-types-set-frozenset)を用いて，包含関係を [`issubset()`](https://docs.python.org/ja/3/library/stdtypes.html#frozenset.issubset) で調べる。
  - ただし，この場合，最終状態だけで評価しているので，途中で決着がついているものも含まれる。

In [5]:
s = {0, 1, 2} # 勝利条件の一つ
m = {0, 1, 2, 3, 4} # X の一連の手番 (moves by X)
s.issubset(m)

True

{0, 1, 2} は set((0, 1, 2)) と同等

In [6]:
{0, 1, 2} == set((0, 1, 2))

True

シーケンス seq に一つでも True があれば [`any(seq)`](https://docs.python.org/ja/3/library/functions.html#any) は True

In [7]:
seq = [False, True, False] 
any(seq)

True

`np.logical_or(a, b)` は bool 型の配列 `a`，`b` について要素ごとの論理和をとった結果の配列を作る。

In [8]:
a = np.array([False, False, True, True])
b = np.array([False, True, False, True])
c = np.logical_or(a, b)
print(repr(c))

array([False,  True,  True,  True])


bool型の配列 x に対して [`np.count_nonzero(x)`](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) は 評価値が True である x の要素の個数を返す。

In [9]:
import numpy as np
x = np.array([True, True, False, False, False, False])
print('Number of elements of x with their value being True: ', np.count_nonzero(x))

Number of elements of x with their value being True:  2


[any(s.issubset(m) for s in win_seq) for m in moves_by_x] は[ジェネレータ式とリスト内包表記](https://docs.python.org/ja/3.8/howto/functional.html#generator-expressions-and-list-comprehensions)の両方を使っている

In [10]:
any((not b) for b in (True, True, False))

True

- リスト内包表記(list comprehension)

In [11]:
['<' + s + '>' for s in ('A', 'B', 'C')]

['<A>', '<B>', '<C>']

```Python
idx_won_by_x = [any(s.issubset(m) for s in win_seq) for m in moves_by_x]
idx_won_by_o = [any(s.issubset(m) for s in win_seq) for m in moves_by_o]
```
の結果として，`idx_won_by_x` は `histories` と同じサイズを持ち，

任意の i $\in \{0, \ldots, n \}$ ($n = $ `len(histories)`) について  
  「`idx_won_by_x[i]` が `True`」 ならば 「`histories[i]` は X が勝利している状態」

が成立する。

In [12]:
for i, won_by_x in enumerate(idx_won_by_x):
    if won_by_x:
        separate_and_print_board(histories[i])
        break

for i, won_by_o in enumerate(idx_won_by_o):
    if won_by_o:
        separate_and_print_board(histories[i])
        break

h =  (0, 1, 2, 3, 4, 5, 6, 7, 8)
m_by_x =  (0, 2, 4, 6, 8)
m_by_o =  (1, 3, 5, 7)
X O X 
O X O 
X O X 

h =  (0, 1, 2, 3, 5, 4, 6, 7, 8)
m_by_x =  (0, 2, 5, 6, 8)
m_by_o =  (1, 3, 4, 7)
X O X 
O O X 
X O X 



上記は，numpy.ndarray の [boolean array indexing](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing) 機能を用いれば，より短く記述できる。

In [13]:
arr_hist = np.array(histories)
separate_and_print_board(arr_hist[idx_won_by_x][0])
separate_and_print_board(arr_hist[idx_won_by_o][0])

h =  [0 1 2 3 4 5 6 7 8]
m_by_x =  [0 2 4 6 8]
m_by_o =  [1 3 5 7]
X O X 
O X O 
X O X 

h =  [0 1 2 3 5 4 6 7 8]
m_by_x =  [0 2 5 6 8]
m_by_o =  [1 3 4 7]
X O X 
O O X 
X O X 



[boolean array indexing](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing) 
の他の例

In [14]:
a = np.arange(1, 7)
idx_even = ((a % 2) == 0) # 偶数ならば True，奇数ならば False
print(repr(a))
print(repr(idx_even))
print(repr(a[idx_even]))

array([1, 2, 3, 4, 5, 6])
array([False,  True, False,  True, False,  True])
array([2, 4, 6])


---

#### ありうる全てのゲームの結果の個数を数える
- 最終的なボードの状態が同じでも、途中の手番が異なれば別の結果として数える
- ゲームのルールに適合する状態を全て列挙するため，再帰呼び出しを利用する (以下の `play()` を参照)
- ゲームの結果を全て記録し，GUI で表示する。表示に関しては，最終的なボードの状態が同じであれば，
  途中の手番が異なっていても同一視する

In [29]:
import itertools as it
import functools
import copy
import ipywidgets
import IPython.display


NUM_CELLS = 9
NUM_COLS = 3
statistics = {'won by X': 0, 'won by O': 0, 'draw': 0}
board_dict = {'won by X': [], 'won by O': [], 'draw': []}

GAME_OVER = 0
ABNORMAL_GAME = 1
GAME_CONTINUED = 2

LEAST_NUM_MOVES_TO_WIN = 5
MAX_NUM_EMPTY_CELLS_TO_WIN = NUM_CELLS - LEAST_NUM_MOVES_TO_WIN

# {'won by X': 131184, 'won by O': 77904, 'draw': 46080}
NUM_OUTCOMES = 131184 + 77904 + 46080
progress_count = it.count()
progress_bar = ipywidgets.IntProgress(
    value=0,
    min=0,
    max=NUM_OUTCOMES,
    description='Counting:',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
PROG_BAR_UPDATE_PERIOD = 1000

# progress bar の状態を更新する
def update_progress_bar():
    count = next(progress_count)
    if count % PROG_BAR_UPDATE_PERIOD == 0: # to avoid delay due to GUI updates
        progress_bar.value = count

    if count == NUM_OUTCOMES:
        progress_bar.value = progress_bar.max


# ゲームの決着がついているか判定する
def game_status(histories):
    final_states = [ 
        {0, 1, 2}, # horizontal
        {3, 4, 5},
        {6, 7, 8},

        {0, 3, 6}, # vertical
        {1, 4, 7},
        {2, 5, 8},

        {0, 4, 8}, # diagonal
        {2, 4, 6},
    ]
    symbol = ['X', 'O']
    if len(histories[0]) == len(histories[1]):
        last_move = 1
        next_move = 0
    else:
        last_move = 0
        next_move = 1

    for s in final_states:
        if s.issubset(histories[last_move]):
            # print(f'{symbol[last_move]} won the game')
            statistics[f'won by {symbol[last_move]}'] += 1
            list_hist = [list(h) for h in histories]
            board_dict[f'won by {symbol[last_move]}'].append(list_hist)

            return GAME_OVER
        if s.issubset(histories[next_move]):
            print('abnormal game status was detected')
            raise RuntimeError('abnormal game status was detected')
            # return ABNORMAL_GAME

    # draw
    if (len(histories[0]) + len(histories[1])) == NUM_CELLS:
        # print('Draw')
        statistics['draw'] += 1
        board_dict['draw'].append(histories)
        return GAME_OVER

    return GAME_CONTINUED



# histories に対応するゲームの状態を表示する
def print_board(histories):
    # print(histories[0])
    # print(histories[1])
    txt = ''
    for i in range(NUM_CELLS):
        symbol = ' '
        if i in histories[0]:
            symbol = 'X'
        elif i in histories[1]:
            symbol = 'O'
        txt += symbol + ' '
        if i % NUM_COLS == (NUM_COLS - 1):
            txt += '\n'
    print(txt)



# ありうる全てのゲームの結果を列挙する
def play(histories, empty_cells):
    n_empty_cells = len(empty_cells)

    if n_empty_cells <= MAX_NUM_EMPTY_CELLS_TO_WIN:
        if game_status(histories) == GAME_OVER:
            # print_board(histories) 
            update_progress_bar()
            return

    if True: # to iterate over all the possible moves (including duplicate states)
        n_cells_to_play = 1
        move = (n_empty_cells + 1) % 2

    else: # quick way to count unique terminal states
        if n_empty_cells == NUM_CELLS:
            n_cells_to_play = 3
            move = 0
        elif n_empty_cells == NUM_CELLS - 3:
            n_cells_to_play = 2
            move = 1
        else:
            n_cells_to_play = 1
            # move = (len(empty_cells) + 1) % 2
            move = (n_empty_cells + 1) % 2

    for c in it.combinations(empty_cells, n_cells_to_play):
        tmp_histories = copy.deepcopy(histories)
        tmp_histories[move] = tmp_histories[move].union(c)
        tmp_empty_cells = empty_cells - set(c)

        play(tmp_histories, tmp_empty_cells)



# 'Show' ボタンまたは 'Board ID' ボックスの値が増減されたときのコールバック関数
def on_click(board_list, txt_box, output, btn):
    output.clear_output(wait=True)
    id = txt_box.value
    with output:
        # print(btn.description)
        print(board_list[id])
        print_board(board_list[id])


if __name__ == '__main__':
    histories = [set(), set()] # for player 'X' and 'O'
    empty_cells = set(range(NUM_CELLS))

    IPython.display.display(progress_bar)

    play(histories, empty_cells)

    print(statistics) # This will show {'won by X': 131184, 'won by O': 77904, 'draw': 46080}


    # 途中の手番のみが異なるが最終状態としては重複している結果を除去する
    # (回転や鏡映の関係にある結果は別々の結果として扱う)
    unique_board_dict = {}
    for k in board_dict.keys():
        if False:
            sorted_hist = [[sorted(h[0]), sorted(h[1])] for h in board_dict[k]]
            unique_board_dict[k] = list({str(v): v for v in sorted_hist}.values())
        else:
            sorted_hist = tuple((tuple(sorted(h[0])), tuple(sorted(h[1]))) for h in board_dict[k])
            unique_board_dict[k] = list(set(sorted_hist))


    if True:
        bd = unique_board_dict
    else:
        bd = board_dict

    vboxes = []
    for k in bd.keys():
        max_id = len(bd[k]) - 1
        lbl = ipywidgets.Label(value=k + f': {max_id + 1} distinct outcomes')
        txt = ipywidgets.BoundedIntText(value=0, min=0, max=max_id, description='Board ID')
        btn = ipywidgets.Button(description='Show')
        out = ipywidgets.Output()
        vboxes.append(ipywidgets.VBox((lbl, txt, btn, out)))

        on_click_n = functools.partial(on_click, bd[k], txt, out)
        btn.on_click(on_click_n)
        txt.observe(on_click_n)


    IPython.display.display(ipywidgets.HBox(vboxes)) 



IntProgress(value=0, bar_style='info', description='Counting:', max=255168)

{'won by X': 131184, 'won by O': 77904, 'draw': 46080}


##### 解説 
- 以下の解説のセルを正常に実行するには Notebook の最初から順にセルを実行しておく必要がある。

`histories[0]` は X のプレイヤーの手番， `histories[1]` は O のプレイヤーの手番を表す。

In [16]:
histories = [set(), set()]
histories[0] = {2, 5, 8}
histories[1] = {0, 3}
print_board(histories)

O   X 
O   X 
    X 



`game_status()` はゲームの決着がついているか判別する

In [17]:
game_status(histories) == GAME_OVER

True

itertools.combinations(t, r) でシーケンスt から r 個の要素を取り出すときにありうる2項組合せを全て生成できる

In [18]:
import itertools as it
t = ('A', 'B', 'C') 
print(tuple(it.combinations(t, 2)))

(('A', 'B'), ('A', 'C'), ('B', 'C'))


[`set` 型](https://docs.python.org/ja/3/library/stdtypes.html#set-types-set-frozenset)に対して 演算子 `-` を用いて部分集合の除去を行える

In [19]:
s1 = {1, 2, 3, 4, 5}
s2 = {2, 3}
print(repr(s1 - s2))

{1, 4, 5}


set型の s1, s2 について s1.union(s2) で s1 と s2 の和集合を作成できる

In [20]:
s1 = {1, 2}
s2 = {99, 100, 101}
print(repr(s1.union(s2)))
print(repr(s1)) # s1 自体は変更されない

{1, 2, 99, 100, 101}
{1, 2}


[`copy.deepcopy(t)`](https://docs.python.org/ja/3/library/copy.html) は代入と異なり，t の要素が参照であれば参照先のオブジェクトをコピーする。
したがって，
```
    tmp_histories = copy.deepcopy(histories)
```
の後に，`tmp_histories` に対する操作を行っても，`histories` には影響しない。

In [21]:
import copy
s1 = {1, 2, 3}
s2 = {4, 5, 6}
t1 = (s1, s2)
t2 = t1
t3 = copy.deepcopy(t1)

print('Before modification')
print('t1: ', repr(t2))
print('t2: ', repr(t2))
print('t3: ', repr(t3))

s1.add(99)
print('After modification')
print('t1: ', repr(t2))
print('t2: ', repr(t2))
print('t3: ', repr(t3))

Before modification
t1:  ({1, 2, 3}, {4, 5, 6})
t2:  ({1, 2, 3}, {4, 5, 6})
t3:  ({1, 2, 3}, {4, 5, 6})
After modification
t1:  ({99, 1, 2, 3}, {4, 5, 6})
t2:  ({99, 1, 2, 3}, {4, 5, 6})
t3:  ({1, 2, 3}, {4, 5, 6})


board_dict は列挙したゲームの結果を保持する

In [22]:
# 先頭の5種類の結果を表示する
for board in it.islice(board_dict['won by X'], 5):
    print_board(board)


X O X 
O X X 
X O O 

X O X 
O X   
X   O 

X O X 
O X O 
X X O 

X O X 
O X O 
    X 

X O X 
O X O 
X     



[`sorted(t)`](https://docs.python.org/ja/3/howto/sorting.html) は t の要素を昇順に並べなおした `list` となる。

In [23]:
t = (5, 2, 4, 1, 3)
print(repr(sorted(t)))

[1, 2, 3, 4, 5]


set によって重複したデータを除去できる。

In [24]:
t = ((1, 2), (3, 4), (1, 2), (3, 4))
print(repr(set(t)))

{(1, 2), (3, 4)}


- set の要素が tuple の場合，tuple の構成要素が同じでも，並び順が異なれば，別の tuple として扱われる。
- set の構築において要素が重複しているか否か(等価性)の判定は，要素の [ハッシュ値](https://docs.python.org/ja/3.8/reference/datamodel.html#object.__hash__) によって行われる。 
- ある二つの tuple の構成要素が同じでも，並び順が異なれば，それぞれに異なるハッシュ値を持つ。
- ハッシュ値を算出できるオブジェクトは [ハッシュ可能](https://docs.python.org/ja/3/glossary.html#term-hashable) と呼ばれる。

In [25]:
t2 = ((1, 1), 
      (1, 1), 
      (2, 3),
      (3, 2), 
      (3, 3))

s2 = set(t2) 
print(s2)

print('\n'.join(f't2[{i}]: {e}, hash(t2[{i}]) == {hash(e)}' for i, e in enumerate(t2)))
print(hash(t2[0]) == hash(t2[1]))
print(hash(t2[2]) == hash(t2[3]))

{(2, 3), (1, 1), (3, 3), (3, 2)}
t2[0]: (1, 1), hash(t2[0]) == 8389048192121911274
t2[1]: (1, 1), hash(t2[1]) == 8389048192121911274
t2[2]: (2, 3), hash(t2[2]) == 8409376899596376432
t2[3]: (3, 2), hash(t2[3]) == 3863035679738500442
t2[4]: (3, 3), hash(t2[4]) == 5972319052856130739
True
False


list は[ハッシュ不可能 (unhashable)](https://docs.python.org/ja/3/glossary.html#term-hashable) なので，set によってデータの重複を除去する方法には使えない。

In [26]:
t = ((1, 2), (3, 4), (1, 2), (3, 4))
l = [list(e) for e in t]
print(l)
print(repr(set(l))) # set(l) は TypeError を起こす

[[1, 2], [3, 4], [1, 2], [3, 4]]


TypeError: unhashable type: 'list'

順序を無視してデータの重複を除外するには，sorted と set を組み合わせる。

In [ ]:
t = ((1, 2), (2, 1), (3, 4), (4, 3))
print(repr(set(t)))
t2 = tuple(tuple(sorted(e)) for e in t)
print(repr(t2))
print(repr(set(t2)))

{(1, 2), (2, 1), (3, 4), (4, 3)}
((1, 2), (1, 2), (3, 4), (3, 4))
{(1, 2), (3, 4)}


VBox, HBox による widget (GUI の部品)の配置

[ipywidgets.Output](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html) を
[コンテキストマネージャ](https://docs.python.org/ja/3/reference/compound_stmts.html#with)
として使うと，print() の結果は Output の領域に表示される。

In [ ]:
import ipywidgets
import functools
import numpy as np
import matplotlib.pyplot as plt

out1 = ipywidgets.Output()
out2 = ipywidgets.Output()
btn1 = ipywidgets.Button(description='Show')
btn2 = ipywidgets.Button(description='Show')
txt1 = ipywidgets.BoundedIntText(value=0, min=0, max=10, description='ID')
txt2 = ipywidgets.BoundedIntText(value=0, min=0, max=10, description='ID')

th = np.linspace(0, 2*np.pi, 100)
px = np.cos(th)
py = np.sin(th)

def on_click(txtbox, output, btn):
    output.clear_output(wait=True) # wait=True でちらつき防止

    # output をコンテキストマネージャとして使うと，print() の結果や
    # matplotlib による plot の結果は output の領域に表示される
    with output:
    # if True:
        # print(btn.description)
        print(txtbox.value)
        plt.plot(px, py)
        plt.gca().set_aspect('equal')
        plt.show()

on_click1 = functools.partial(on_click, txt1, out1)
on_click2 = functools.partial(on_click, txt2, out2)
btn1.on_click(on_click1)
btn2.on_click(on_click2)

box1 = ipywidgets.VBox((txt1, btn1, out1))
box2 = ipywidgets.VBox((txt2, btn2, out2))
ipywidgets.HBox((box1, box2))

進捗状況を示すバーの表示方法

In [ ]:
import ipywidgets
import IPython.display

progress_bar = ipywidgets.IntProgress(
    value=7,
    min=0,
    max=10,
    description='Counting:',
    # bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    # style={'bar_color': 'maroon'},
    # orientation='horizontal'
)
IPython.display.display(progress_bar)

progress_bar.value = 5 

IntProgress(value=7, bar_style='info', description='Counting:', max=10)

[`functools.partial()`](https://docs.python.org/ja/3/howto/functional.html#the-functools-module) で関数の一部の引数を固定した関数を作成できる

In [ ]:
import functools

def foo(a, b):
    return a + b

f1 = functools.partial(foo, 1)
f1(2) # foo(1, 2) を呼び出すことと同等


3

itertools.count() は 0, 1, 2, ... を順に返す[イテレータ](https://docs.python.org/ja/3/glossary.html#term-iterator)を作る。  
イテレータから順に要素を取り出すには組み込み関数の [next()](https://docs.python.org/ja/3/library/functions.html#next) を用いる

In [ ]:
import itertools as it
gen = it.count()
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
1
2
3


#### 参考文献
- [ipywidgets のドキュメント](https://ipywidgets.readthedocs.io/en/stable/index.html)
- [functools のドキュメント](https://docs.python.org/3/library/functools.html)